In [0]:
import pandas as pd
import numpy as np
import seaborn as sns

In [2]:
df = sns.load_dataset('tips')
df.sample(5)

,total_bill,tip,sex,smoker,day,time,size
240,27.18,2.00,Female,Yes,Sat,Dinner,2
133,12.26,2.00,Female,No,Thur,Lunch,2
234,15.53,3.00,Male,Yes,Sat,Dinner,2
171,15.81,3.16,Male,Yes,Sat,Dinner,2
16,10.33,1.67,Female,No,Sun,Dinner,3


In [4]:
df.groupby('sex').mean()

,total_bill,tip,size
sex,,,
Male,20.744076,3.089618,2.630573
Female,18.056897,2.833448,2.459770


In [5]:
df['prot_tip'] = df['tip']/df['total_bill']
df.head(3)

,total_bill,tip,sex,smoker,day,time,size,prot_tip
0,16.99,1.01,Female,No,Sun,Dinner,2,0.059447
1,10.34,1.66,Male,No,Sun,Dinner,3,0.160542
2,21.01,3.50,Male,No,Sun,Dinner,3,0.166587


In [7]:
df.groupby('sex').aggregate({'prot_tip': ['mean', 'median']})

prot_tip          
            mean    median
sex                       
Male    0.157651  0.153492
Female  0.166491  0.155581

In [8]:
df.groupby('sex')[['prot_tip']].describe()

prot_tip                      ...                              
          count      mean       std  ...       50%       75%       max
sex                                  ...                              
Male      157.0  0.157651  0.064778  ...  0.153492  0.186240  0.710345
Female     87.0  0.166491  0.053632  ...  0.155581  0.194266  0.416667

[2 rows x 8 columns]

In [10]:
def mean_eur2usd(x):
  return np.mean(x) * 1.12

df.groupby('sex')[['total_bill', 'prot_tip']].apply(mean_eur2usd)

,total_bill,prot_tip
sex,,
Male,23.233366,0.176569
Female,20.223724,0.186470


In [12]:
dict_agg = {
    'tip': [min, max],
    'total_bill': [np.mean, mean_eur2usd],
}

# .agg is equal that .aggregate
df.groupby(['sex', 'time']).agg(dict_agg)

tip        total_bill             
                min    max       mean mean_eur2usd
sex    time                                       
Male   Lunch   1.44   6.70  18.048485    20.214303
       Dinner  1.00  10.00  21.461452    24.036826
Female Lunch   1.25   5.17  16.339143    18.299840
       Dinner  1.00   6.50  19.213077    21.518646

In [18]:
def f_filter(x):
  return mean_eur2usd(x['total_bill']) > 20

df.groupby(['sex', 'time']).filter(f_filter).head(5)

,total_bill,tip,sex,smoker,day,time,size,prot_tip
0,16.99,1.01,Female,No,Sun,Dinner,2,0.059447
1,10.34,1.66,Male,No,Sun,Dinner,3,0.160542
2,21.01,3.50,Male,No,Sun,Dinner,3,0.166587
3,23.68,3.31,Male,No,Sun,Dinner,2,0.139780
4,24.59,3.61,Female,No,Sun,Dinner,4,0.146808
